In [ ]:
../julia-1.10.0/bin/julia --project=. --thread 32

In [ ]:
using HopfieldDCA; import Enzyme: gradient;
filepath = "../DataAttentionDCA/data/PF00014/PF00014_mgap6.fasta.gz"; H = 2; alg = HopPlmVar(H, filepath);tmp = HopfieldDCA.StgArr(alg); _w = alg.W ./ sum(alg.W); KK = zeros(alg.N,alg.N,H);
#gradient((p1,p2)->get_loss_J_tmp(p1,p2, alg.Z, _w, tmp, KK), alg.K, alg.V)
n_ep = 3; η = 0.5; λ = 0.001;
@time m,n = trainer_J(alg, n_ep, η = η, λ = λ);

n_ep = 3; η = 0.5; λ = 0.001;
@time m,n = trainer(alg, n_ep, η = η, λ = λ);
gradient((p1,p2)->get_loss_J_tmp(p1,p2, alg.Z, _w, tmp, KK), alg.K, alg.V)
n_ep = 3; η = 0.5; λ = 0.001;
@time m,n = trainer_J(alg, n_ep, η = η, λ = λ);
@time m,n = trainer_J(alg, n_ep, η = η, λ = λ, 
    savefile="log_J/H$(H)η$(η)λ$(λ)T$(n_ep).txt");
tmp = HopfieldDCA.StgArr(alg); _w = alg.W ./ sum(alg.W);
@time get_loss_J(alg.K, alg.V, alg.Z, _w);
@time get_loss_J_tmp(alg.K, alg.V, alg.Z, _w, tmp)



println("Done")
@time m,n = trainer(alg, n_ep, η = η, λ = λ, 
    savefile="log/H$(H)η$(η)λ$(λ)T$(n_ep).txt");


In [ ]:
#code for analytical gradient
using HopfieldDCA; using Tullio; import Flux: gradient;
filepath = "../DataAttentionDCA/data/PF00014/PF00014_mgap6.fasta.gz"; H = 2; alg = HopPlmVar_gen(H, filepath); _w = alg.W ./ sum(alg.W); @tullio delta_j[a, j, m] := a == alg.Z[j, m] (a in 1:alg.q); tmp = Stg(alg);
@time gt = get_anal_grad(alg.K, alg.V, alg.Z, _w, tmp, delta_j);
@time gk,gv = gradient((p1,p2)->get_loss_J(p1,p2, alg.Z, _w), alg.K, alg.V);
sum(gt.K[:].-gk[:])
sum(gt.V[:].-gv[:])
import BenchmarkTools: @btime
@btime gkt,gvt = get_anal_grad(alg.K, alg.V, alg.Z, _w, tmp, delta_j);
@btime gk,gv = gradient((p1,p2)->get_loss_J(p1,p2, alg.Z, _w), alg.K, alg.V);
n_ep = 3; η = 0.5; λ = 0.001;
@time m,n = trainer_J(alg, n_ep, η = η, λ = λ);
@time m,n = trainer(alg, n_ep, η = η, λ = λ);



In [ ]:
structfile = "../DataAttentionDCA/data/PF00014/PF00014_struct.dat"
s = HopfieldDCA.score(m.K, m.V); PPV_h = HopfieldDCA.compute_PPV(s, structfile);

close("all"); plot(ppv_plm[1], label="Plm"); plot(PPV_h, label="HopPlm");legend();savefig("ciao.png");

